# Classification Analysis

The following notebook was created to understand how vendors classify the samples, possibility to understand the amount of FP & FN and to try to normalize the used naming.

In [42]:
import numpy as np
import pandas as pd
from IPython.display import display

samples = pd.read_csv('data/av_classes.csv', dtype=str)
samples['date'] = pd.to_datetime(samples['date'], format='%Y/%m/%d')
# Set date as index
samples = samples.set_index('date')

Checking the presence (in percentage) of each vendor in the samples. Helps identifying useful vendors as well as wrong data parsing.

In [2]:
vendors_presence = dict()
size = len(samples)
for col in samples.columns[2:]:
    vendors_presence[col] = samples[col].count() / size
    
vendors_presence = pd.Series(data=vendors_presence)    
print('Total samples: {0}\n'.format(len(samples)))
print('Vendor presence:')
display(vendors_presence)

Total samples: 68150

Vendor presence:


 win32_virut_xp (v)     0.000015
a-squared               0.024637
ad-aware                0.830565
aegislab                0.719809
agnitum                 0.859178
ahnlab                  0.000778
ahnlab-v3               0.966559
alibaba                 0.324109
alyac                   0.362627
antivir                 0.551005
antivir7                0.000044
antiy-avl               0.979296
arcabit                 0.247161
authentium              0.030110
avast                   0.993485
avast5                  0.016581
avg                     0.993133
avira                   0.421761
avware                  0.433236
baidu                   0.089773
baidu-international     0.829464
bitdefender             0.999061
bkav                    0.842348
bytehero                0.858811
cat-quickheal           0.999266
clamav                  0.996097
cmc                     0.787616
command                 0.000924
commtouch               0.518782
comodo                  0.993514
          

The following gives a rough estimate for FP & FN rate per vendor.

The approach is limited since it's based on the majority, meaning if more than 50% classify as clean, it's probably clean (likewise for non clean). The problem rises when a vendor is able detect malware before all others, as such, FP rate might be skewed.

In [44]:
# Filter vendors with more than XX% presence
high_vendors = vendors_presence[vendors_presence >= 0.01]
# display(high_vendors)

# filtered = samples
filtered = samples[samples.link == 'MzIwZDgzMjY0YjQ5NGQxMjhkZjk1YjE0YTlkNGQ1OTE']
display(filtered)

cols = high_vendors.keys().tolist()

filtered = samples.filter(cols)
vendors_fp = dict.fromkeys(high_vendors.keys().tolist(), 0)
vendors_fn = dict.fromkeys(high_vendors.keys().tolist(), 0)

for _, s in filtered.iterrows():
    break
    vendor_count = s.count()
    # For unknown samples
    if vendor_count == 0:
        continue
    s = s.dropna()
    clean_count = s[s == 'clean'].count()
    
    # Majority says it's clean (or they're all wrong and the minority is right)
    is_clean = True if clean_count / vendor_count >= 0.5 else False
    if is_clean and clean_count / vendor_count < 1.0:
        print(_)
        print(s)
        break
    
    for v, c in s.iteritems():
        # False Positive
        if is_clean and c != 'clean':
            vendors_fp[v] += 1
        # False Negative
        elif is_clean == False and c == 'clean':
            vendors_fn[v] += 1

print(counter)
vendors_fp = pd.Series(data=vendors_fp)
vendors_fn = pd.Series(data=vendors_fn)

# print(vendors_fp)
# print(vendors_fn)

,md5,link,win32_virut_xp (v),a-squared,ad-aware,aegislab,agnitum,ahnlab,ahnlab-v3,alibaba,...,trendmicro-housecall,una,vba32,vipre,virobot,virusbuster,webwasher-gateway,yandex,zillya,zoner
date,,,,,,,,,,,,,,,,,,,,,


63044


In [37]:
display(vendors_fn[vendors_fn == 0])

alibaba    0
alyac      0
arcabit    0
avira      0
avware     0
cyren      0
esafe      0
pctools    0
zoner      0
dtype: int64